<a href="https://colab.research.google.com/github/jdan98/bib/blob/master/IBM_final_task_2_data_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Capstone Project - Open the best japanease restaurant in paris

I love paris I love eat and i love japanease. This is a good list to make a exercice about it !!!

**Our Problem**

In this project we will try to find an optimal location for opening an Japanease restaurant in Paris.

Since there are lots of japanease restaurants in Paris we will try to detect **locations with no Japanease restaurants** in vicinity. We would also prefer **locations as close to city center as possible**.

Let's meet these two conditions ! 

Based on definition of our problem, factors that will influence our decission are:

- number of existing restaurants in the neighborhood (any type of restaurant)
- number of and distance to Japanease restaurants in the neighborhood, if any
distance of neighborhood from city center
We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API
coordinate of Paris center will be obtained using Google Maps API geocoding of well known Paris location (Chatelet les halles => Eglise Saint Eustache)


Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around paris city center.

Let's first find the latitude & longitude of paris city center, using specific, well known address and Google Maps geocoding API.

In [ ]:
!pip install folium
!pip install geopy
#https://geopy.readthedocs.io/en/stable/#installation
#https://colab.research.google.com/drive/19TbFZCaOBcCJ8aNoRM0CAfhAZqDnPaVm#scrollTo=iLH4mCcn2Ybw

In [ ]:
from geopy.geocoders import Nominatim
def get_coordinates(adress):
  geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")
  location = geolocator.geocode(adress)
  return[location.latitude, location.longitude]
  

#address = 'Alexanderplatz, Germany'
address = "Eglise Saint-Eustache"
paris_center= get_coordinates(address)
print('Coordinate of {}: {}'.format(address, paris_center))

#Create a geocoder that uses the OpenStreetMap Nominatim API
#geocodes = [geolocator.geocode(', '.join([df['Landmark'][i], df['City'][i], df['State'][i]]))for i in range(len(df))]

Coordinate of Eglise Saint-Eustache: [48.8633997, 2.3451272318983762]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~4,5km from Eglise Saint Eustache. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [ ]:
!pip install shapely
!pip install pyproj

     |████████████████████████████████| 10.9MB 2.8MB/s 


In [ ]:
import shapely.geometry
import pyproj
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('paris center longitude={}, latitude={}'.format(paris_center[1], paris_center[0]))
x, y = lonlat_to_xy(paris_center[1], paris_center[0])
print('paris center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('paris center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
paris center longitude=2.3451272318983762, latitude=48.8633997
paris center UTM X=-427073.62116932566, Y=5489970.097454985
paris center longitude=2.345127231898378, latitude=48.86339970000001


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
paris_center_x, paris_center_y = lonlat_to_xy(paris_center[1], paris_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = paris_center_x - 6000
x_step = 600
y_min = paris_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(paris_center_x, paris_center_y, x, y)
        if (distance_from_center <= 4501):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stabl

208 candidate neighborhood centers generated.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stab

In [ ]:
import folium
map_paris = folium.Map(location=paris_center, zoom_start=13)
folium.Marker(paris_center, popup='Alexanderplatz').add_to(map_paris)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_paris) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_paris)
    #folium.Marker([lat, lon]).add_to(map_paris)
map_paris

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~4,5km from Alexanderplatz.

Let's now use Geopy to get approximate addresses of those locations.

In [ ]:
def get_address(latitude, longitude):
  location = geolocator.reverse("{},{}".format(latitude,longitude))
  return location

addr = get_address(paris_center[0], paris_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(paris_center[0], paris_center[1], addr))

Reverse geocoding check
-----------------------
Address of [48.8633997, 2.3451272318983762] is: Église Saint-Eustache, Rue de Turbigo, Quartier des Halles, Quartier Les Halles, Paris, Île-de-France, France métropolitaine, 75001, France


In [ ]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat,lon)
    if address is None:
        address = 'NO ADDRESS'
    address_str = str(address)
    #address = address.replace(', France', '') # We don't need country part of address
    addresses.append(address_str)
    print(' .', end='')
print('\n done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 done.


In [ ]:
addresses[1:10]

['49, Rue Boussingault, Cité Florale, Quartier de la Maison-Blanche, Paris, Île-de-France, France métropolitaine, 75013, France',
 '7, Rue Martin Bernard, Cité Florale, Quartier de la Maison-Blanche, Paris, Île-de-France, France métropolitaine, 75013, France',
 '4, Rue Toussaint-Féron, Cité Florale, Paris 13e Arrondissement, Paris, Île-de-France, France métropolitaine, 75013, France',
 'Crèche Collective Municipale, Rue Sthrau, Paris 13e Arrondissement, Paris, Île-de-France, France métropolitaine, 75013, France',
 'Restaurant universitaire CROUS Tolbiac, Rue Leredde, Quartier de la Gare, Paris, Île-de-France, France métropolitaine, 75013, France',
 '45, Rue des Plantes, Quartier du Petit-Montrouge, Paris 14e Arrondissement, Paris, Île-de-France, France métropolitaine, 75014, France',
 '26, Rue du Commandeur, Quartier du Petit-Montrouge, Paris 14e Arrondissement, Paris, Île-de-France, France métropolitaine, 75014, France',
 'GHU Paris Psychiatrie & neurosciences - site Sainte-Anne, 1, R

In [ ]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Réservoir de Montsouris, Avenue René Coty, Qua...",48.824688,2.334530,-428573.621169,5.485813e+06,4419.275959
1,"49, Rue Boussingault, Cité Florale, Quartier d...",48.825579,2.342506,-427973.621169,5.485813e+06,4253.234064
2,"7, Rue Martin Bernard, Cité Florale, Quartier ...",48.826469,2.350482,-427373.621169,5.485813e+06,4167.733197
3,"4, Rue Toussaint-Féron, Cité Florale, Paris 13...",48.827358,2.358458,-426773.621169,5.485813e+06,4167.733197
4,"Crèche Collective Municipale, Rue Sthrau, Pari...",48.828247,2.366435,-426173.621169,5.485813e+06,4253.234064
5,"Restaurant universitaire CROUS Tolbiac, Rue Le...",48.829135,2.374412,-425573.621169,5.485813e+06,4419.275959
6,"45, Rue des Plantes, Quartier du Petit-Montrou...",48.827912,2.321398,-429473.621169,5.486333e+06,4357.751714
7,"26, Rue du Commandeur, Quartier du Petit-Montr...",48.828803,2.329374,-428873.621169,5.486333e+06,4058.324778
8,GHU Paris Psychiatrie & neurosciences - site S...,48.829694,2.337350,-428273.621169,5.486333e+06,3830.143600
9,"Quartier de Croulebarbe, Paris, Île-de-France,...",48.830584,2.345326,-427673.621169,5.486333e+06,3686.461718


In [ ]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

Coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. We will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'japanease restaurant' category, as we need info on Japanease restaurants in the neighborhood.

In [ ]:
# Category IDs corresponding to japanease restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
import requests
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

#4bf58dd8d48988d1d2941735 #sushi

japanease_restaurant_categories = ['4bf58dd8d48988d111941735', '55a59bace4b013909087cb0c', '55a59bace4b013909087cb30',
                             '55a59bace4b013909087cb21', '55a59bace4b013909087cb24',
                             '55a59bace4b013909087cb27', '4bf58dd8d48988d1d2941735', '55a59bace4b013909087cb2a']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, radius=500, limit=100):
    client_secret = 'KDTVGQ22RMGJ5DM0MN5IOC1BA1ETRF5UQXDLM15E4GXKYEJC'
    client_id = 'P1QPGC4MCBCHYNIL3MNR04RJN4HISMTVU3OQH4CCBILMXW43'
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        print("ok")
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                  get_categories(item['venue']['categories']),
                  (item['venue']['location']['lat'], item['venue']['location']['lng']),
                  format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        print('pute')
        venues = []
    return venues

In [ ]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found japanease restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    japanease_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        print('a')
        venues = get_venues_near_location(lat, lon, food_category,radius=350, limit=100)
        print ('b')
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_japanease = is_restaurant(venue_categories, specific_filter=japanease_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_japanease, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_japanease:
                    japanease_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, japanease_restaurants, location_restaurants

restaurants = {}
japanease_restaurants = {}
location_restaurants = []
restaurants, japanease_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)


Obtaining venues around candidate locations:a
ok
b
 .a
ok


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


b
 .a
ok


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/go

b
 .a
ok


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/go

b
 .a
ok
b
 .a
ok


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/go

b
 . done.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of japanease restaurants:', len(japanease_restaurants))
print('Percentage of japanease restaurants: {:.2f}%'.format(len(japanease_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 3508
Total number of japanease restaurants: 367
Percentage of japanease restaurants: 10.46%
Average number of restaurants in neighborhood: 14.716346153846153


In [ ]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4ca4d10eb0ff8cfa3811c35e', "Il Far'niente", 48.82362335218033, 2.330605664073317, '1 rue Paul Fort, 75014 Paris, France', 311, False, -428880.621370311, 5485743.793684114)
('537670d0498e07dd3a603084', 'Kazoku', 48.826068293079146, 2.331854842523171, '105 rue de la Tombe Issoire, 75014 Paris, France', 249, True, -428743.43334727467, 5485999.155138713)
('4c43f416f97fbe9aa38fba30', 'Assanabel', 48.827553329411536, 2.3356957658324973, "6 rue d'Alésia (Rue du Saint-Gothard), 75014 Paris, France", 267, False, -428434.69627976895, 5486116.125979744)
('4c38dd221e06d13aca92783e', 'Shinotaku', 48.826423643937055, 2.341066896915436, "1 bis rue d'Alésia, 75014 Paris, France", 141, True, -428063.0375059026, 5485924.561068291)
('4be9af92246fb71301d1abdf', 'Pizza Caravelle', 48.82669515682725, 2.3392416461501853, "5 bis rue d'Alésia, 75014 Paris, France", 269, False, -428191.44420109084, 5485977.208143253)
('51718f83498ebe088fd1efb5', 'LatinoGourmand'

In [ ]:
print('List of japanease restaurants')
print('---------------------------')
for r in list(japanease_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(japanease_restaurants))

List of japanease restaurants
---------------------------
('537670d0498e07dd3a603084', 'Kazoku', 48.826068293079146, 2.331854842523171, '105 rue de la Tombe Issoire, 75014 Paris, France', 249, True, -428743.43334727467, 5485999.155138713)
('4c38dd221e06d13aca92783e', 'Shinotaku', 48.826423643937055, 2.341066896915436, "1 bis rue d'Alésia, 75014 Paris, France", 141, True, -428063.0375059026, 5485924.561068291)
('4c06aaf60ed3c928fa63787d', 'Oh Sushi!', 48.828555115519706, 2.3428630623948266, '99 rue de la Glacière, 75013 Paris, France', 332, True, -427891.7584930012, 5486138.451683138)
('4dc19611d4c07bbdf778c990', 'Le Feu de Mars', 48.82765336333092, 2.3532366239172404, '41 rue Vandrezanne, 75013 Paris, France', 241, True, -427149.97456186404, 5485910.363737242)
('4b8d6ce7f964a52006fb32e3', 'Sushi Ichi', 48.827330280406066, 2.3564637284021783, '1 rue du Moulinet, 75013 Paris, France', 146, True, -426920.0027100642, 5485834.715663164)
('4b87b422f964a520e9c731e3', 'Tokyo Choisy', 48.828615

In [ ]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Garance, Restaurant Le Concorde, Da Rocco, Le Bourbon, Chez Françoise, Le Concorde, Casa Germain, Aux Ducs de Bourgogne
Restaurants around location 102: Le Restaurant du Musée d'Orsay, Le Poulpry, Le Vin de Bellechasse
Restaurants around location 103: LouLou, Nodaiwa, Sanukiya, La Rotonde St. Honoré, La Guinguette des Tuileries, Bistrot Mee, Zen, Higuma
Restaurants around location 104: Le Louis, Le Pot de Vins, Boutique yam'Tcha, Baltard Au Louvre, Kei, La Régalade Saint-Honoré, Olio Pane Vino, Les Fines Gueules
Restaurants around location 105: Raviolis Chinois Nord-Est, La Maison du Saké, Tir-Bouchon, Pirouette, Rice & Fish, Pupetta, Petit Bao, Street Bangkok Roast & Beer
Restaurants around location 106: Le Lac de l'Ouest, Taing Song-Heng, La Rusidda, Le Derrière, Kitchen, Ai's Bento, Le 404, La Massara
Restaurants around location 107: L'Îlot, Banoï, L'Aller Retour, Margo, Goku Asian Canteen, Cuis

In [ ]:
map_paris = folium.Map(location=paris_center, zoom_start=13)
folium.Marker(paris_center, popup='Saint E').add_to(map_paris)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_japanease = res[6]
    color = 'red' if is_japanease else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_paris)
map_paris

Looking good. So now we have all the restaurants in area within few kilometers and we know which ones are Jap restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

In this project we will direct our efforts on detecting areas of paris that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required data: location and type (category) of every restaurant within 6km from paris center. We have also identified Italian restaurants (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of paris - we will use heatmaps to identify a few promising areas close to center with low number of restaurants in general (and no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step, we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Japanease restaurants in radius of 400 meters. 

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the number of restaurants in every area candidate:

In [ ]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 14.716346153846153


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Jap restaurant
0,"Réservoir de Montsouris, Avenue René Coty, Qua...",48.824688,2.334530,-428573.621169,5.485813e+06,4419.275959,1,251.842402
1,"49, Rue Boussingault, Cité Florale, Quartier d...",48.825579,2.342506,-427973.621169,5.485813e+06,4253.234064,5,142.835648
2,"7, Rue Martin Bernard, Cité Florale, Quartier ...",48.826469,2.350482,-427373.621169,5.485813e+06,4167.733197,24,243.851092
3,"4, Rue Toussaint-Féron, Cité Florale, Paris 13...",48.827358,2.358458,-426773.621169,5.485813e+06,4167.733197,36,141.649645
4,"Crèche Collective Municipale, Rue Sthrau, Pari...",48.828247,2.366435,-426173.621169,5.485813e+06,4253.234064,9,294.039243
5,"Restaurant universitaire CROUS Tolbiac, Rue Le...",48.829135,2.374412,-425573.621169,5.485813e+06,4419.275959,11,121.552298
6,"45, Rue des Plantes, Quartier du Petit-Montrou...",48.827912,2.321398,-429473.621169,5.486333e+06,4357.751714,7,251.113375
7,"26, Rue du Commandeur, Quartier du Petit-Montr...",48.828803,2.329374,-428873.621169,5.486333e+06,4058.324778,13,18.011987
8,GHU Paris Psychiatrie & neurosciences - site S...,48.829694,2.337350,-428273.621169,5.486333e+06,3830.143600,3,349.778055
9,"Quartier de Croulebarbe, Paris, Île-de-France,...",48.830584,2.345326,-427673.621169,5.486333e+06,3686.461718,5,214.471647


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [ ]:
distances_to_japanease_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in japanease_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_japanease_restaurant.append(min_distance)

df_locations['Distance to Jap restaurant'] = distances_to_japanease_restaurant
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Jap restaurant
0,"Réservoir de Montsouris, Avenue René Coty, Qua...",48.824688,2.334530,-428573.621169,5.485813e+06,4419.275959,1,251.842402
1,"49, Rue Boussingault, Cité Florale, Quartier d...",48.825579,2.342506,-427973.621169,5.485813e+06,4253.234064,5,142.835648
2,"7, Rue Martin Bernard, Cité Florale, Quartier ...",48.826469,2.350482,-427373.621169,5.485813e+06,4167.733197,24,243.851092
3,"4, Rue Toussaint-Féron, Cité Florale, Paris 13...",48.827358,2.358458,-426773.621169,5.485813e+06,4167.733197,36,141.649645
4,"Crèche Collective Municipale, Rue Sthrau, Pari...",48.828247,2.366435,-426173.621169,5.485813e+06,4253.234064,9,294.039243
5,"Restaurant universitaire CROUS Tolbiac, Rue Le...",48.829135,2.374412,-425573.621169,5.485813e+06,4419.275959,11,121.552298
6,"45, Rue des Plantes, Quartier du Petit-Montrou...",48.827912,2.321398,-429473.621169,5.486333e+06,4357.751714,7,251.113375
7,"26, Rue du Commandeur, Quartier du Petit-Montr...",48.828803,2.329374,-428873.621169,5.486333e+06,4058.324778,13,18.011987
8,GHU Paris Psychiatrie & neurosciences - site S...,48.829694,2.337350,-428273.621169,5.486333e+06,3830.143600,3,349.778055
9,"Quartier de Croulebarbe, Paris, Île-de-France,...",48.830584,2.345326,-427673.621169,5.486333e+06,3686.461718,5,214.471647


In [ ]:
print('Average distance to closest Jap restaurant from each area center:', df_locations['Distance to Jap restaurant'].mean())

Average distance to closest Jap restaurant from each area center: 258.9820255507917


OK, so on average Jap restaurant can be found within ~1km from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's create a map showing heatmap / density of restaurants and try to extract some meaningfull info from that. Also, let's show few circles indicating distance of 1km, 2km and 3km from Alexanderplatz.

In [ ]:

restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

japanease_latlons = [[res[2], res[3]] for res in japanease_restaurants.values()]

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

map_paris = folium.Map(location=paris_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_paris) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_paris)
folium.Marker(paris_center).add_to(map_paris)
folium.Circle(paris_center, radius=1000, fill=False, color='white').add_to(map_paris)
folium.Circle(paris_center, radius=2000, fill=False, color='white').add_to(map_paris)
folium.Circle(paris_center, radius=3000, fill=False, color='white').add_to(map_paris)
map_paris

This map is not so 'hot' (Italian restaurants represent a subset of ~15% of all restaurants in Berlin) but it also indicates higher density of existing Italian restaurants directly north and west from Alexanderplatz, with closest pockets of low Italian restaurant density positioned east, south-east and south from city center.

Based on this we will now focus our analysis on areas south-west, south, south-east and east from Berlin center - we will move the center of our area of interest and reduce it's size to have a radius of 2.5km. This places our location candidates mostly in boroughs Kreuzberg and Friedrichshain (another potentially interesting borough is Prenzlauer Berg with large low restaurant density north-east from city center, however this borough is less interesting to stakeholders as it's mostly residental and less popular with tourists).


OK. Now let's calculate two most important things for each location candidate: number of restaurants in vicinity (we'll use radius of 250 meters) and distance to closest Italian restaurant.

In [ ]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

restaurant_counts = []
japanease_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(xs, ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, japanease_restaurants)
    japanease_distances.append(distance)
print('done.')

Generating data on location candidates... done.


In [ ]:
# Let's put this into dataframe
df_locations_2 = pd.DataFrame({'Latitude':latitudes,
                                 'Longitude':longitudes,
                                 'X':xs,
                                 'Y':ys,
                                 'Restaurants nearby':restaurant_counts,
                                 'Distance to Japanease restaurant':japanease_distances})

df_locations_2.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Japanease restaurant
0,48.824688,2.334530,-428573.621169,5.485813e+06,0,251.842402
1,48.825579,2.342506,-427973.621169,5.485813e+06,4,142.835648
2,48.826469,2.350482,-427373.621169,5.485813e+06,18,243.851092
3,48.827358,2.358458,-426773.621169,5.485813e+06,24,141.649645
4,48.828247,2.366435,-426173.621169,5.485813e+06,3,294.039243
5,48.829135,2.374412,-425573.621169,5.485813e+06,8,121.552298
6,48.827912,2.321398,-429473.621169,5.486333e+06,4,251.113375
7,48.828803,2.329374,-428873.621169,5.486333e+06,9,18.011987
8,48.829694,2.337350,-428273.621169,5.486333e+06,2,349.778055
9,48.830584,2.345326,-427673.621169,5.486333e+06,5,214.471647


OK. Let us now filter those locations: we're interested only in locations with no more than two restaurants in radius of 250 meters, and no Italian restaurants in radius of 400 meters.

In [ ]:
good_res_count = np.array((df_locations_2['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_jap_distance = np.array(df_locations_2['Distance to Japanease restaurant']>=400)
print('Locations with no Japanease restaurants within 400m:', good_jap_distance.sum())

good_locations = np.logical_and(good_res_count, good_jap_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations_2[good_locations]

Locations with no more than two restaurants nearby: 38
Locations with no Japanease restaurants within 400m: 32
Locations with both conditions met: 14


In [ ]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values


map_paris = folium.Map(location=paris_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_paris)
folium.Marker(paris_center).add_to(map_paris)
folium.Circle(paris_center, radius=4500, color='red', fill=False).add_to(map_paris)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_paris) 
map_paris